<a href="https://colab.research.google.com/github/mlfa03/MLOPs/blob/main/Docker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Docker

## CheatSheet

### Docker CLI

#### Run a new container

In [ ]:
docker run IMAGE                                             #Start a new container from an image
docker run --name CONTAINER IMAGE                            # assign the container created in the previous step, a name
docker run -p HOSTPORT:CONTAINERPORT IMAGE                   # map a port 
docker run -p IMAGE                                          # map all ports 
docker run -d IMAGE                                          # run a container in the background 
docker run --hostname HOSTNAME IMAGE                         # assign it a hostname
docker run --add-host HOSTNAME:IP IMAGE                      # add a dns entry 
docker run -v HOSTDIR:TARGETDIR IMAGE                        # map a local directory into the container 
docker run -it --entrypoint EXECUTABLE IMAGE                 # but change the entrypoint 

In [ ]:
#Example of the commands above:
docker run nginx
docker run --name web nginx
docker run -p 8080:80 nginx
docker run -p nginx
docker run -d nginx 
docker run --hostname srv nginx 
docker run --add-host HOSTNAME:IP IMAGE 
docker run -v ~/:/usr/share/nginx/html nginx 
docker run -it --entrypoint bash nginx 

#### Manage containers

In [ ]:
docker ps                                                  # show a list of running containers
docker ps -a                                               # show a list of all containers
docker rm CONTAINER                                        # Delete a container
docker rm -f CONTAINER                                     # Delete a running container
docker container prune                                     # Delete stopped containers
docker start CONTAINER                                     # Start a stopped container
docker cp CONTAINER:SOURCE TARGET                          # Copy a file from a container to the host
docker cp TARGET CONTAINER:SOURCE                          # Copy a file from a host to a container
docker exec -it CONTAINER EXECUTABLE                       # Start a shell inside a running container
docker rename OLD_NAME NEW_NAME                            # Rename a container
docker commit CONTAINER                                    # Create an image out of a container

In [ ]:
#Example of the commands above:
docker rm web
docker rm -f web
docker stop web
docker start web
docker cp web:/index.html index.html
docker cp index.html web:/index.html
docker exec -it web bash 
docker rename 096 web
docker commit web

#### Manage Images

In [ ]:
docker pull IMAGE[:TAG]                                                # Download an Image
docker push IMAGE                                                      # Upload an image to a repository
docker rmi IMAGE                                                       # Delete an image
docker images                                                          # Show a list of all images
docker image prune                                                     # Delete dangling images
docker image prune -a                                                  # Delete all unused images
docker build DIRECTORY                                                 # Build an image from a dockerfile
docker tag IMAGE NEWIMAGE                                              # Tag an image
docker build -t IMAGE DIRECTORY                                        # Build and tag an image from a Dockerfile
docker save IMAGE > FILE                                               # Save an image to a tar file
docker load -i TARFILE                                                 # Load an image from a tar file

In [ ]:
#Examples from commands above:
docker pull nginx
docker push myimage:1 0
docker build .
docker tag ubuntu ubuntu:18.04
docker build -t myimage
docker save nginx > nginx.tar
docker load -i nginx.tar 

#### Info & Stats

In [ ]:
docker logs CONTAINER                                                   # Show the logs of a container
docker stats                                                            # Show stats of running containers
docker version                                                          # Show installed docker version 
docker inspect NAME                                                     # get detailed info about object
docker diff CONTAINER                                                   # Show all modified files in container
docker port CONTAINER                                                   # Show mapped ports of a container


## Basics

### What is containerization?

A runtime instance of a docker image contains 3 things:

* A docker image
* An execution environment 
* A standard set of instructions 

**Docker Engine**

* Comprised of the runtime and packaging tool 
* Must be installed on the host that runs docker

**Docker Store**
* Online cloud service where users can store and share their docker images 
* Also known as docker hub 

## Installation and configuration 

### Installation

In [ ]:
sudo apt-get remove docker docker-engine docker-ce docker.io                                #Uninstall Docker
sudo apt-get update                                                                         # Update the apt package index
sudo apt-get install \                                                                      # Allow apt to use a repository over HTTPS
    apt-transport-https \
    ca-certificates \
    curl \
    software-properties-common

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -                # Add Docker’s official GPG key to apt
sudo apt-key fingerprint 0EBFCD88                                                           # Add Docker’s official GPG key to apt
sudo add-apt-repository \                                                                   # Add the Docker repository to apt
   "deb [arch=amd64] https://download.docker.com/linux/ubuntu \
   $(lsb_release -cs) \
   stable"

sudo apt-get update                                                                         # Re-Update the apt package index now that we have the Docker repositories added

In [ ]:
sudo apt-get install docker-ce=17.12.0~ce-0~ubuntu                                       # Install a specific version of Docker
sudo groupadd docker                                                                     # Make sure that the Docker group is already added
sudo usermod -aG docker $USER                                                            # Add your username to the Docker group

# Log out and back in

# Test Docker with the version command and hello-world
docker version
docker run hello-world

### UCP (Universal Control Plane)

Docker hosts all running the Docker Engine? How do you manage a Docker infrastructure at that type of scale? That's what Docker's Enterprise Edition provides you. It includes the Universal Control Plane and the Docker Trusted Registry

In [ ]:
# Install UCP
docker container run --rm -it --name ucp \
  -v /var/run/docker.sock:/var/run/docker.sock \
  docker/ucp:2.2.5 install \
  --host-address <node-ip-address> \
  --interactive

### Backup UCP

In [ ]:
# Backup UCP

docker container run \
  --log-driver none --rm \
  --interactive \
  --name ucp \
  -v /var/run/docker.sock:/var/run/docker.sock \
  docker/ucp:2.2.6 backup \
  --id <ucp-instance-id> \
  --passphrase "secret" > /tmp/backup.tar

# Backup registry content data

sudo tar -cf backup-images.tar \ $(dirname $(docker volume inspect --format '{{.Mountpoint}}' dtr-registry-<replica-id>))

# Backup DTR metadata

read -sp 'ucp password: ' UCP_PASSWORD; \
docker run --log-driver none -i --rm \ 
--env UCP_PASSWORD=$UCP_PASSWORD \
docker/dtr:2.3.11 backup \ 
--ucp-url <ucp-url> \
 --ucp-insecure-tls \
 --ucp-username <ucp-username> \
 --existing-replica-id <replica-id> > backup-metadata.tar

In [ ]:
docker run -it --rm docker/dtr:2.4.12 install --ucp-insecure-tls

### Docker storage

 It's always recommended to store data inside Docker volumes and get that data outside of containers, as those provide better performance and isolation. However, if you have a work load that needs persistent data in the writable layer of your container, you should look at storage drivers

In [ ]:
# To find out what storage driver you are using 
docker info


Instructions on how to change storage drivers: https://docs.docker.com/storage/storagedriver/select-storage-driver/

### Creating a repository and pushing it to docker hub

A Docker repository is a collection of related Docker images with the same name but different tags. A registry, on the other hand, is a place where you store your images. A registry can be public, it can be private, and it can have multiple repositories. The most common and default registry for Docker is Docker's own public registry, Docker Hub.

In [ ]:
docker login                                   #login to docker hub
docker pull IMAGE                              #Pulling an image
docker images                                  #to see the images information
docker tag Xyz my_username/REPO_NAME:my_tag    #Giving a tag to an image and sending to your repository on docker hub (xyz are 3 first letters of image tag)
docker push my_username/REPO_NAME              #push to docker hub
docker login                                   #login to docker hub

## Orchestration 

### Getting started with docker swarm

The swarm cluster is run by what we call managers and the work is done by worker nodes, which run the swarm services. 

In [ ]:
docker swarm init                             #initializing docker swarm
docker swarm join --token [xxxxxxx]           #to add a worker to the swarm
docker swarm join-token manager 

#Now the cluster is created, we can check it:
docker node ls                                #visualize node in the cluster
docker swarm join-token worker                #to see the token of the worker 
docker swarm join --token [token copied]  
#"this node joined swarm as a worker" message appears

**Nodes**
The node is your physical host or could it be a virtual machine in the Cloud that's running docker, the docker engine


**Services**
They are automatically run across however many nodes are needed based on the number of replicas that you defined

**Containers**
Runs on one host


**Tasks**

In [ ]:
docker service create --name name_of_service --replicas=number-of-replicas IMAGE   #creating a service

docker service ls                                                                  #checking if the service was created

 If you're creating a brand new Docker swarm cluster, you can enable autolock when you initialize swarm with this command: docker swarm init --autolock

In [ ]:
#Locking a swarm cluster

docker swarm init --autolock                            #locking swarm cluster when creating it 
docker swarm update --autolock=true                     #locking an existent swarm cluster

#You need to restart docker before doing this and also need to unlock the cluster
sudo systemctl restart docker 
docker node ls 
docker swarm unlock

### Manage docker swarm 

In [ ]:
docker info | more                                  #get more information on the cluster
docker service ls                                   #
docker inspect

In [ ]:
#Stack files
cat docker-stack.yml                                          #visualize yaml stack file 
docker stack deploy --compose-file docker-stack.yml mystack   #deploying stack file 
docker stack ls                                               #visualizing


#Manipulate running stack of services
docker stack ps mystack                                      #visualizing the stack we deployed 
cat docker-stack.yml #and make modifications directly in the file
#rerun docker stack deploy --compose-file docker-stack.yml mystack to update 

### Troubleshooting docker swarm

In [ ]:
docker service logs

docker service ls            #see services that are running 

## Image Creation, Management and Registry 

, an image is an executable package that includes everything needed to run an application, including the code, a runtime, libraries, environmental variables and configuration files

## Storage and Volumes

## Networking

## Security 

**Docker content trust**

* Docker notary is an open source project that provides the ability to digitally sign content and is also included with DTR
* Enforces a policy that any operations with a remote registry require signed images 

ENABLED WITH: export DOCKER_CONENT_TRUST=1

* Once enabled, the docker push will try to sign an image and the docker pull will only download signed images

### Configuring

**Docker access control model**
* subject (user, team, entire organization, service account)
* roles (defines what can be done)
* resources (swarm collections, kubernetes namespaces)
* grants (

###

### Configure docker networking

In [ ]:
docker network ls                                       #visualize network id
docker network inspect bridge                           # inspect bridge network  (bridge is default network)
docker network create                                   #create a user defined network
docker network create --driver bridge app-net           #create a user defined network
docker network ls                                       #visualize network id

### Troubleshooting docker networking
